<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/Agents_with_OpenAI_Assistants_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai==1.37.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 4.3 MB/s eta 0:00:00


In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 tiktoken==0.7.0 --force-reinstall

In [3]:
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

# Math Tutor


In [4]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
)

In [5]:
thread = client.beta.threads.create()

In [6]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

In [7]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [8]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [9]:
print(messages[0].content[0].text.value)

The solution to the equation \(3x + 11 = 14\) is \(x = 1\).


# Customer Support


In [10]:
!wget https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf

--2025-01-21 14:24:17--  https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf
Resolving personales.unican.es (personales.unican.es)... 193.144.193.111
Connecting to personales.unican.es (personales.unican.es)|193.144.193.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267816 (262K) [application/pdf]
Saving to: ‘LinuxCommandLineCheatSheet_1.pdf’

LinuxCommandLineChe 100%[===================>] 261.54K   406KB/s    in 0.6s    

2025-01-21 14:24:18 (406 KB/s) - ‘LinuxCommandLineCheatSheet_1.pdf’ saved [267816/267816]



In [11]:
from openai import OpenAI

client = OpenAI()

In [12]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Tech Support")

# Ready the files for upload to OpenAI
file_streams = [open("LinuxCommandLineCheatSheet_1.pdf", "rb")]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


One of the important aspects of this process is that the OpenAI service extracts information from PDFs, indexes it, converts it into embeddings, and retrieves data. This service simplifies the entire process, which is beneficial if you are not comfortable with coding. However, it can be limiting since you don’t control the underlying operations.

In [13]:
assistant = client.beta.assistants.create(
    name="Tech Support",
    instructions="You are a tech support chatbot. Use the product manual to respond accurately to customer inquiries.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [14]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What 'ls' command do?",
        }
    ]
)

In [15]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [16]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

print(messages[0].content[0].text.value)

The `ls` command in Linux is used to list the contents of a directory. It displays the files and directories within the specified directory. When used with the `-al` option, it lists all files, including hidden ones, in a long listing format that provides detailed information【4:1†LinuxCommandLineCheatSheet_1.pdf】.


In [17]:
messages[0].content[0].text.annotations

[FileCitationAnnotation(end_index=314, file_citation=FileCitation(file_id='file-DhPcD3WdTqugJZt5NmZdSw'), start_index=276, text='【4:1†LinuxCommandLineCheatSheet_1.pdf】', type='file_citation')]